In [3]:
import pandas as pd # type: ignore
import requests # type: ignore
from bs4 import BeautifulSoup # type: ignore

In [4]:
years = range(2010, 2025)
dfs = []
for year in years:

    url = f'https://www.basketball-reference.com/players/c/curryst01/gamelog/{year}'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, features = 'html.parser')

    table = soup.find_all('table')[7]

    header_row = table.find('thead')
    column_titles = header_row.find_all('th')

    column_titles_cleaned = [title.text for title in column_titles]

    column_titles_cleaned.pop(0)
    column_titles_cleaned[4] = 'Loc'
    column_titles_cleaned[6] = 'PointSpread'
    # column_titles_cleaned.append("Season")  #changed

    df = pd.DataFrame(columns = column_titles_cleaned)

    body = table.find('tbody')
    body_data = body.find_all('tr')
    for row in body_data:
        row_data = row.find_all('td')
        individual_row_data = [data.text.strip() for data in row_data]
        if len(individual_row_data) != 0 and individual_row_data[0] != '':    #filters out the rows of repeat column header info and the games steph didn't play
            df.loc[len(df)] = individual_row_data
    df.insert(1, "Season", [year] * len(df))

    df.replace('', pd.NA, inplace=True)
    dfs.append(df)
    df.to_csv(rf'C:\Users\Drew Brackman\Documents\Portfolio\Stephen Curry Stats\curry{year}.csv')

In [5]:
merged_df = pd.concat((df for df in dfs), ignore_index= True)
merged_df.to_csv(r'C:\Users\Drew Brackman\Documents\Portfolio\Stephen Curry Stats\currycareer.csv')

In [30]:
from matplotlib import pyplot as plt


# plt.scatter(merged_df.G, merged_df.PTS)
# plt.show()
# 
fifty_plus_games = merged_df[pd.to_numeric(merged_df.PTS) >= 50]
fifty_plus_games